In [1]:
pip install -U nemo_toolkit['asr']


In [5]:
from nemo.collections.asr.models import ASRModel


In [7]:

asr_ast_model = ASRModel.from_pretrained(model_name="nvidia/canary-1b-v2")


canary-1b-v2.nemo:  90%|########9 | 5.71G/6.36G [00:00<?, ?B/s]

[NeMo I 2025-08-16 17:42:33 nemo_logging:393] Tokenizer CanaryBPETokenizer initialized with 16384 tokens


[NeMo W 2025-08-16 17:42:33 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_lhotse: true
    skip_missing_manifest_entries: true
    input_cfg: null
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    num_workers: 4
    pin_memory: true
    prompt_format: canary2
    max_duration: 40.0
    min_duration: 0.01
    text_field: answer
    lang_field: target_lang
    use_bucketing: true
    max_tps: null
    bucket_duration_bins: null
    bucket_batch_size: null
    num_buckets: null
    bucket_buffer_size: 20000
    shuffle_buffer_size: 10000
    
[NeMo W 2025-08-16 17:42:33 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the 

[NeMo I 2025-08-16 17:42:33 nemo_logging:393] PADDING: 0
[NeMo I 2025-08-16 17:42:46 nemo_logging:393] Model EncDecMultiTaskModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--canary-1b-v2/snapshots/7cc2afaad76086004e89254b2de1d4cc7dcdc69f/canary-1b-v2.nemo.


In [9]:
!wget https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav


--2025-08-16 17:43:10--  https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
Resolving dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)... 3.5.130.168, 3.5.131.60, 52.219.93.186, ...
Connecting to dldata-public.s3.us-east-2.amazonaws.com (dldata-public.s3.us-east-2.amazonaws.com)|3.5.130.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 237964 (232K) [audio/wav]
Saving to: ‘2086-149220-0033.wav’

2086-149220-0033.wa 100%[===================>] 232.39K   365KB/s    in 0.6s    

2025-08-16 17:43:11 (365 KB/s) - ‘2086-149220-0033.wav’ saved [237964/237964]



In [14]:
import wave

with wave.open("2086-149220-0033.wav", "rb") as wav_file:
    channels = wav_file.getnchannels()
    sample_width = wav_file.getsampwidth()
    framerate = wav_file.getframerate()
    n_frames = wav_file.getnframes()
    duration = n_frames / float(framerate)

    print(f"Channels: {channels}")
    print(f"Sample Width: {sample_width} bytes")
    print(f"Frame Rate (Hz): {framerate}")
    print(f"Number of Frames: {n_frames}")
    print(f"Duration (s): {duration:.2f}")


Channels: 1
Sample Width: 2 bytes
Frame Rate (Hz): 16000
Number of Frames: 118960
Duration (s): 7.43


In [17]:
import soundfile as sf

info = sf.info("2086-149220-0033.wav")
print(info)

2086-149220-0033.wav
samplerate: 16000 Hz
channels: 1
duration: 7.435 s
format: WAV (Microsoft) [WAV]
subtype: Signed 16 bit PCM [PCM_16]


In [12]:
output = asr_ast_model.transcribe(['2086-149220-0033.wav'], source_lang='en', target_lang='en')
print(output[0].text)


[NeMo W 2025-08-16 17:44:09 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: trim_silence
[NeMo W 2025-08-16 17:44:09 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the training speed if your tokenizer is very large.If the impact is noticable, set pretokenize=False in dataloader config.(note: that will disable token-per-second filtering and 2D bucketing features)
Transcribing: 1it [00:00,  2.06it/s]

Well, I don't wish to see it any more, observed Phoebe, turning away her eyes. It is certainly very like the old portrait.


In [18]:
output = asr_ast_model.transcribe(['2086-149220-0033.wav'], source_lang='en', target_lang='lt')
print(output[0].text)

[NeMo W 2025-08-16 17:48:29 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: trim_silence
[NeMo W 2025-08-16 17:48:29 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the training speed if your tokenizer is very large.If the impact is noticable, set pretokenize=False in dataloader config.(note: that will disable token-per-second filtering and 2D bucketing features)
Transcribing: 1it [00:00,  1.76it/s]

Na, aš nenoriu jo matyti, pastebėjo Phoebe, nukreipusi žvilgsnį. Jis tikrai labai panašus į senąjį portretą.


In [23]:
from pydub import AudioSegment

audio = AudioSegment.from_file("test2.m4a")
audio = audio.set_channels(1)     # convert to mono
audio = audio.set_frame_rate(16000)  # resample to 16 kHz
audio.export("test_mono2.wav", format="wav")


<_io.BufferedRandom name='test_mono2.wav'>

In [24]:
output = asr_ast_model.transcribe(['test_mono2.wav'], source_lang='lt', target_lang='lt')
print(output[0].text)

[NeMo W 2025-08-16 18:07:35 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: trim_silence
[NeMo W 2025-08-16 18:07:35 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the training speed if your tokenizer is very large.If the impact is noticable, set pretokenize=False in dataloader config.(note: that will disable token-per-second filtering and 2D bucketing features)
Transcribing: 1it [00:01,  1.22s/it]

Garys Grožės salonas apsilankė šiuo metu ir kelia klausimą, kodėl negalima susikalbėti lietujo kalbą ir ar gali būti, kad dėl to paslaugos kainuoja pigiau. Kirkiai ir Grožės specialistų asociacijos prezidentas sakė pastebinti, jog kartais užsieniečiai dempinguoja kainęs ir tikime, kad sektoriui šiuo metu negalba.


## Whisper

In [25]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

In [26]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [27]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)


Device set to use cuda:0


In [29]:
result = pipe("test_mono.wav")
print(result["text"])

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


 Dalys grožio salonuose apsilanktusių lietuvių kelia klausimų. Paliau nevejama susikalbėti lietuvių kalbą ir ar gali būti, kad dėl to paslaugos kainuoja pigiau? Kirpėjai ir grožio specialistų asociacijos prezidentė sakė paspebentį, jo kartais užsinečiai dempinguoja kainas ir tikina, kad sektoriui šiuo metu nevengo.


In [32]:
result = pipe("test_mono2.wav", generate_kwargs={"language": "lithuanian"})
print(result["text"])

You have passed language=lithuanian, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=lithuanian.


 Darys grožio salonas apsilankęs šiuo metu, jo kelia klausimą, kodėl negalima susikalbėti be tojo kalbą ir gali būti, kad dėl to paslaugos kainuoja pigiau. Kirpėjai ir grožio specialisto asociacijos prezidentas sankė pastebinti, jo kartais užsieniečiai dėmpingojo kainės ir tikina, kad sektoriai šiuo metu ne ganga.
